In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('ggplot')

In [3]:
trainFile=pd.read_csv("gender_age_train.csv")
testFile=pd.read_csv("gender_age_test.csv")

In [4]:
#print(trainFile.shape)#(74654,4)
trainFile[0:1]

,device_id,gender,age,group
0,-8076087639492063270,M,35,M32-38


In [5]:
#print(testFile.shape)#(112071,1)
testFile[0:1]

,device_id
0,1002079943728939269


In [6]:
#Understanding Target Value Characteristics(Regression w.r.t. Age, Classification w.r.t. Gender)
trainGrpBy=trainFile.groupby("group")
trainGroups=trainGrpBy.groups
trainGroupDict=[(key,len(trainGroups[key])) for key in trainGroups.keys()]
print(trainGroupDict)
#Female Groups are : F23-,F24-26,F27-28,F29-32,F33-42,F43+
#Male Groups are : M22-,M23-26,M27-28,M29-31,M32-38,M39+

[('F43+', 4194), ('M27-28', 5445), ('M29-31', 7309), ('M23-26', 9605), ('F24-26', 4190), ('F29-32', 4628), ('F27-28', 3118), ('F33-42', 5561), ('M32-38', 9476), ('M39+', 8581), ('M22-', 7488), ('F23-', 5050)]


In [7]:
len(trainFile.groupby('device_id').groups.keys())#Number of unique device_ids: 74645 i.e. All device_id are unique

74645

In [8]:
trainFile.describe()#Interesting we have a Kid of Age 1 and grandpa having age 96 using Mobile Device :-)
#But Most of the users are withing age group 25 to 36

,device_id,age
count,7.464500e+04,74645.000000
mean,-7.491354e+14,31.410342
std,5.327150e+18,9.868735
min,-9.223067e+18,1.000000
25%,-4.617367e+18,25.000000
50%,-1.841362e+16,29.000000
75%,4.636656e+18,36.000000
max,9.222849e+18,96.000000


In [9]:
eventsData=pd.read_csv("events_subset.csv")
print(eventsData[0:2])
#eventsData.describe()
#event_id is PK here;device_id may get duplicated
#longitude: -180 to 151 : Continuous
#latitude: -37 to 53.64 : Continuous

   event_id            device_id            timestamp  longitude  latitude
0         1    29182687948017175  2016-05-01 00:55:25     121.38     31.24
1         2 -6401643145415154744  2016-05-01 00:54:12     103.65     30.97


In [10]:
#As expected => Very High Correlation between longitude and latitude : 0.947287
eventsData.corr()

,event_id,device_id,longitude,latitude
event_id,1.000000,0.001050,-0.003879,-0.004633
device_id,0.001050,1.000000,-0.013765,-0.007659
longitude,-0.003879,-0.013765,1.000000,0.947287
latitude,-0.004633,-0.007659,0.947287,1.000000


In [11]:
app_eventsData=pd.read_csv("app_events_subset.csv")

In [12]:
app_eventsData[0:1]

,event_id,app_id,is_installed,is_active
0,2,5927333115845830913,1,1


In [21]:
appDataGrpByObj=app_eventsData.groupby('app_id')
eventsDataGrpByObj=app_eventsData.groupby('event_id')
isActiveGrpByObj=app_eventsData.groupby('is_active')
isInstalledGrpByObj=app_eventsData.groupby('is_installed')
app_event_GrpByObj=app_eventsData.groupby(['app_id','event_id'])
print(len(app_eventsData),len(appDataGrpByObj.groups),len(eventsDataGrpByObj.groups),
      len(app_event_GrpByObj.groups),len(isActiveGrpByObj.groups),len(isInstalledGrpByObj.groups))
#No Single PK for the Table app_events: (app_id,event_id) together makes the PK
#2 Categorical Variables : is_active and is_installed
#2 Continuous but Identity Columns : app_id and event_id

99999 5504 3553 99999 2 1


In [23]:
app_labelsData=pd.read_csv("app_labels_subset.csv")
app_labelsData[0:1]

,app_id,label_id
0,7324884708820027918,251


In [29]:
app_idGrpByObj=app_labelsData.groupby('app_id')
label_idGrpByObj=app_labelsData.groupby('label_id')
print(len(app_idGrpByObj.groups),len(label_idGrpByObj.groups),len(app_labelsData))
#No PK for app_labels Table
app_labelsData.describe()
#label_id is between 251 to 691
#app_id is between -9.21e18 to 9.22e18

3502 31 4999


,app_id,label_id
count,4.999000e+03,4999.000000
mean,2.888705e+17,477.645929
std,5.277960e+18,127.849836
min,-9.216547e+18,251.000000
25%,-4.133423e+18,262.000000
50%,6.240908e+16,549.000000
75%,5.191946e+18,549.000000
max,9.220205e+18,691.000000


In [31]:
label_categoriesData=pd.read_csv("label_categories.csv")
label_categoriesData[0:1]

,label_id,category
0,1,NaN


In [38]:
label_categoriesData.describe()#label_id is from 1 to 1021
#Note I am working with Subset of Data: So there can be inconsistencies while trying to find exact matches
categoryGrpByObj=label_categoriesData.groupby('category')
labelidGrpByObj=label_categoriesData.groupby('label_id')
print(len(categoryGrpByObj.groups),len(labelidGrpByObj.groups),len(label_categoriesData))
#label_id is PK for label_categories table
print(categoryGrpByObj.groups)#Mostly there is On-to-One association between categort and label_id; However there
#are few excptions(1:N) like 'Financial Information': [143, 708, 921]
#Thought point: Not sure: If we will like to comeup with another category ourself with more generic categories
# In the subset of data number of categories are 835: Hopefullt their count is almost same for the entire dataset as well

835 930 930
{'China Mainland': [310], 'Makeup': [333], 'Finance and economics': [822], 'blogs': [151], 'Olympic': [800], 'mesasge': [154], 'Western Mythology': [82], 'foreign language': [118], 'Cool trendy': [778], 'Pregnant baby': [760], 'computer': [327], 'game-Finding fault': [6], 'military': [796], 'Financial Supermarket': [706], 'Leisure and entertainment': [353], 'Domestic travel': [848], 'game-pet': [8], 'Pharmacy / medical devices / Health': [394], 'Consumer preferences': [271], 'library': [122], 'poker game_doudizhu': [41], 'Precious Metals': [240], 'fashion brand': [285], 'Housekeeping': [207], 'Financial Futures': [690], 'Home Decoration': [338], 'email': [155], 'Brands': [533], 'knight game': [77], 'Liability': [670], 'Chinese Classical Mythology': [81], 'Ping': [614], 'Location': [362], 'children books': [128], 'Relatives': [496, 779], 'Greece': [396], 'Desktop Enhancements': [226], 'Japan and South Korea (Travel)': [857], 'Digital Accessories': [326], 'news': [139], '5.0-

In [41]:
phone_brand_device_modelData=pd.read_csv("phone_brand_device_model.csv")
phone_brand_device_modelData[0:5]

,device_id,phone_brand,device_model
0,-8890648629457979026,小米,红米
1,1277779817574759137,小米,MI 2
2,5137427614288105724,三星,Galaxy S4
3,3669464369358936369,SUGAR,时尚手机
4,-5019277647504317457,三星,Galaxy Note 2


In [ ]:
#Need to Autoate handling chinese names